In [1]:
# Import dependencies
import numpy as np
import pandas as pd
import datetime

In [ ]:
# Import learning model dependencies
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None
tf.random.set_seed(0)

In [ ]:
# Establish connection to AWS server
from config import db_password
import psycopg2
connection = psycopg2.connect(
    host = 'launch-it-1.cyo6pvehqvyz.us-east-1.rds.amazonaws.com',
    port = 5432,
    user = 'postgres',
    password = db_password,
    database='launch-it-1'
    )
cursor=connection.cursor()

In [ ]:
# Pull SQL dataset from database
sql = """
SELECT * FROM launchit  
"""

# Build dataframe from SQL data
stock_data = pd.read_sql(sql, con=connection)
stock_data.head()

In [ ]:
# Create a function to run the LSTM learning model for each company
def learning_model(df):
    # download the data
    y = df['Adj Close'].fillna(method='ffill')
    y = y.values.reshape(-1, 1)

    # scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(y)
    y = scaler.transform(y)

    # generate the input and output sequences
    n_lookback = 365  # length of input sequences (lookback period)
    n_forecast = 180  # length of output sequences (forecast period)

    X = []
    Y = []

    for i in range(n_lookback, len(y) - n_forecast + 1):
        X.append(y[i - n_lookback: i])
        Y.append(y[i: i + n_forecast])

    X = np.array(X)
    Y = np.array(Y)

    # fit the model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(n_lookback, 1)))
    model.add(LSTM(units=50))
    model.add(Dense(n_forecast))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(X, Y, epochs=3, batch_size=32, verbose=0)

    # generate the forecasts
    X_ = y[- n_lookback:]  # last available input sequence
    X_ = X_.reshape(1, n_lookback, 1)

    Y_ = model.predict(X_).reshape(-1, 1)
    Y_ = scaler.inverse_transform(Y_)

    # organize the results in a data frame
    df_past = df[['Close']].reset_index()
    df_past.rename(columns={'index': 'Date', 'Close': 'Actual'}, inplace=True)
    df_past['Date'] = pd.to_datetime(df_past['Date'])
    df_past['Forecast'] = np.nan
    df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]

    df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
    df_future['Date'] = pd.date_range(start=df_past['Date'].iloc[-1] + pd.Timedelta(days=1), periods=n_forecast)
    df_future['Forecast'] = Y_.flatten()
    df_future['Actual'] = np.nan

    results = df_past.append(df_future).set_index('Date')
    
    # Calculate rate of return on investment based on projected future stock price
    return_rate = (results['Forecast'][-1] - df['Adj Close'][-1]) / df['Adj Close'][-1]
    
    return return_rate

    # plot the results - optional
    #results.plot(title='VIST', figsize=(16,8))

In [ ]:
# Temporary cell used for testing output of function to build future dataframe of tickers and growth rates
# *** do not remove until testing complete ***
ticker_df = stock_data[stock_data.TCKR == "BLI"]
ticker_df = ticker_df.set_index('Date')
learning_model(ticker_df)

In [ ]:
# Generate unique list of IPO stock symbols
unique_ipos = stock_data['TCKR'].unique()

# Create new dataframe to house all company stock symbols and their associated growth rates
company_roi_df = pd.DataFrame(unique_ipos, columns=['TCKR'])

# Iterate through each unique stock symbol and filter the historical stock data for each company
for index, ticker in company_roi_df.iterrows():
    ticker = ticker.values[0]
    ticker_df = stock_data[stock_data.TCKR == ticker]
    
    # Reset the index for the filtered dataframe
    ticker_df = ticker_df.set_index('Date')
    
    # Run the learning model function for each company
    company_roi_df.loc[index, 'ROI'] = learning_model(ticker_df)